In [ ]:
import pandas as pd
import numpy as np
import datetime 
import demark_waves
from dateutil.relativedelta import relativedelta


import ipywidgets as widgets
import ipydatagrid as ipd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import bql

In [ ]:
bq = bql.Service()

In [ ]:
fx_pairs = [
    'EURUSD',  # Euro
    'USDJPY',  # Japanese Yen
    'GBPUSD',  # British Pound
    'AUDUSD',  # Australian Dollar
    'USDCAD',  # Canadian Dollar
    'USDCHF',  # Swiss Franc (non-G20 but often included)
    'USDCNH',  # Chinese Yuan    
    'USDMXN',  # Mexican Peso
    'USDZAR',  # South African Rand
    'USDTRY',  # Turkish Lira
    'XAUUSD',  # Physical Gold
    'XBTUSD',  # Bitcoin
]

fx_nominal = {'EURUSD':1000000.00, 'USDJPY':1000000.00, 'GBPUSD':1000000.00, 'AUDUSD':1000000.00, 'USDCHF':1000000.00, 'USDCNH':1000000.00, 'USDMXN':1000000.00, 'USDZAR':1000000.00,
             'USDTRY':1000000.00, 'XAUUSD':500, 'XBTUSD':20}


    

In [ ]:
def get_bql_data(currency, start_date):
    universe = [f'{currency} Curncy']
    
    data_item = {
        'Open': bq.data.px_open(
            dates=bq.func.range(start_date - relativedelta(years=2), today, frq='D'),
            fill='NA'
        ).dropna(),
        'Close': bq.data.px_last(
            dates=bq.func.range(start_date - relativedelta(years=2), today, frq='D'),
            fill='NA'
        ).dropna(),
        'High': bq.data.px_high(
            dates=bq.func.range(start_date - relativedelta(years=2), today, frq='D'),
            fill='NA'
        ).dropna(),
        'Low': bq.data.px_low(
            dates=bq.func.range(start_date - relativedelta(years=2), today, frq='D'),
            fill='NA'
        ).dropna()
    }  
    request = bql.Request(universe, data_item)   
    response = bq.execute(request)
    
    df = bql.combined_df(response).set_index('DATE')
    
    return df

In [ ]:
def createMACDsignals(currency, input_data, fast_span = 7, slow_span = 15, sig_span = 6, start_date = datetime.datetime(1970,1,1), max_blocks = np.Inf):
    data = input_data.copy()

    col_name = 'Close'
    data['macd_fast'] = data[col_name].ewm(span=fast_span).mean()
    data['macd_slow'] = data[col_name].ewm(span=slow_span).mean()
    data['macd'] = (data['macd_fast'] - data['macd_slow'])
    data['macd_sig'] = data['macd'].ewm(span=sig_span).mean()
    data['trade_signal'] = 0
    data['prev_trade_signal'] = 0
    data['trade_number_start'] = 0
    data['trade_number_end'] = 0
    data['trade_pnl'] = 0
    data['blocks'] = 0
    
    rows = data.itertuples()
    prev = next(rows)
    trade_counter = 0
    block_counter = 0
    for line, row in enumerate(rows, 1):        
        
        if pd.to_datetime(row.Index) > start_date:            
            if (block_counter >= max_blocks-1) & np.abs(prev.trade_signal) > 0:
                data.loc[row.Index, 'trade_number_start'] = trade_counter
                row = row._replace(trade_number_start=trade_counter)
        
                data.loc[row.Index, 'trade_signal'] = 0                               
                row = row._replace(trade_signal = 0)
                block_counter = 0                
                
                data.loc[row.Index, 'trade_number_end'] = 0
                row = row._replace(trade_number_start=0)
        
            elif prev.trade_signal == 0:
                data.loc[row.Index, 'trade_number_start'] = 0
                row = row._replace(trade_number_start=0)
                    
                if (row.macd > row.macd_sig) & (prev.macd <= prev.macd_sig) & ((row.WaveBull in ('0', 'P1', 'C1', '3', '5', 'B'))&(row.WaveBear in ('2', '4', 'A', 'C'))):
                    trade_counter += 1
                    block_counter = 0
                
                    data.loc[row.Index, 'trade_signal'] = 1                               
                    row = row._replace(trade_signal = 1)    
                    
                    data.loc[row.Index, 'trade_number_end'] = trade_counter
                    row = row._replace(trade_number_end=trade_counter)
                
                elif (row.macd < row.macd_sig) & (prev.macd >= prev.macd_sig) & ((row.WaveBear in ('0', 'P1', 'C1', '3', '5', 'B'))&(row.WaveBull in ('2', '4', 'A', 'C'))):
                    trade_counter += 1
                    block_counter = 0
                                        
                    data.loc[row.Index, 'trade_signal'] = -1           
                    row = row._replace(trade_signal = -1)
                    data.loc[row.Index, 'trade_number_end'] = trade_counter
                    row = row._replace(trade_number_end=trade_counter)
                else:
                    data.loc[row.Index, 'trade_number_end'] = 0
                    row = row._replace(trade_number_end=0)
                
                
            elif prev.trade_signal == 1:
                data.loc[row.Index, 'trade_number_start'] = trade_counter
                row = row._replace(trade_number_start=trade_counter)
                    
                if (row.macd < row.macd_sig) & (prev.macd >= prev.macd_sig):
                    trade_counter += 1
                    block_counter = 0
                    
                    data.loc[row.Index, 'trade_signal'] = -1           
                    row = row._replace(trade_signal = -1)
                    
                else:
                    data.loc[row.Index, 'trade_signal'] = prev.trade_signal               
                    row = row._replace(trade_signal = prev.trade_signal )
                    
                    block_counter += 1
                
                data.loc[row.Index, 'trade_number_end'] = trade_counter
                row = row._replace(trade_number_end=trade_counter)
                
                
            elif prev.trade_signal == -1:
                data.loc[row.Index, 'trade_number_start'] = trade_counter
                row = row._replace(trade_number_start=trade_counter)
                    
                if (row.macd > row.macd_sig) & (prev.macd <= prev.macd_sig):
                    trade_counter += 1
                    block_counter = 0
                    
                    data.loc[row.Index, 'trade_signal'] = 1            
                    row = row._replace(trade_signal = 1)
                else:
                    data.loc[row.Index, 'trade_signal'] = prev.trade_signal               
                    row = row._replace(trade_signal = prev.trade_signal )
                    
                    block_counter += 1            
                
                data.loc[row.Index, 'trade_number_end'] = trade_counter
                row = row._replace(trade_number_end=trade_counter)
        
        data.loc[row.Index, 'blocks'] = block_counter               
        row = row._replace(blocks = block_counter )        
        
        data.loc[row.Index, 'trade_pnl'] = prev.trade_signal * (getattr(row, 'Close') - getattr(prev,'Close')) * fx_nominal[currency]
        row = row._replace(trade_pnl=prev.trade_signal * (getattr(row, 'Close') - getattr(prev,'Close')) * fx_nominal[currency])
        
        data.loc[row.Index, 'prev_trade_signal'] = prev.trade_signal
        row = row._replace(prev_trade_signal=prev.trade_signal)
        
        prev = row
    
    trade_info = data[['trade_number_start', 'trade_pnl', 'prev_trade_signal']].loc[data.trade_number_start > 0].loc[start_date:].groupby('trade_number_start').agg(total_pnl=('trade_pnl','sum'),
                                                                                                                                               avg_pnl=('trade_pnl','mean'),
                                                                                                                                               stdev_pnl=('trade_pnl','std'),
                                                                                                                                               BuySell=('prev_trade_signal', 'first'))
    data['trade_cum_pnl'] = data.groupby('trade_number_start')['trade_pnl'].cumsum()
    data['total_cum_pnl'] = data['trade_pnl'].cumsum() 
    data['trade_max_cum_pnl'] = data.groupby('trade_number_start')['trade_cum_pnl'].cummax()
    data['trade_draw_down'] = data['trade_cum_pnl'] - data['trade_max_cum_pnl']
    trade_info['Maximum Drawdown'] = data.groupby('trade_number_start')['trade_draw_down'].min()    
    data['my_date'] = pd.to_datetime(data.index)
    trade_info['Start Date'] = data.groupby('trade_number_end')['my_date'].min()
    trade_info['End Date'] = data.groupby('trade_number_start')['my_date'].max()    
    trade_info.index.names = ['Trade Number']   
    trade_info.rename(columns={'total_pnl': 'Total PnL', 'avg_pnl': 'Average PnL', 'stdev_pnl': 'Stdev PnL'}, inplace=True)
    return data, trade_info

In [ ]:
def get_dataframe(currency, start_date):

    df = get_bql_data(currency, start_date)         
    
    return df

In [ ]:
def build_chart(df, currency,start_date):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    #fig1 = px.line(df.loc[start_date:]['Close'])
    fig1 = go.Figure(data=[go.Candlestick(x=df.loc[start_date:].index,
                open=df.loc[start_date:]['Open'],
                high=df.loc[start_date:]['High'],
                low=df.loc[start_date:]['Low'],
                close=df.loc[start_date:]['Close'])])
    
    #fig2 = px.line(df.loc[start_date:]['total_cum_pnl'])
    
    #fig2.update_traces(yaxis="y2")
    #fig.add_traces(fig1.data + fig2.data)
    fig.add_traces(fig1.data)
        
    
    rows = df[start_date:].itertuples()
    
    BullWaveType = '0'   
    BullWaveNumber = -1
    BearWaveType = '0'
    BearWaveNumber = -1
    trade_counter = -1
    
    for line, row in enumerate(rows, 1): 
        if (BullWaveNumber < row.WaveBullNumber):         
            fig.add_annotation(x=row.Index, y=getattr(row,'Close'),text=BullWaveType, showarrow=False, yshift=10, font=dict(color='cyan'))
            BullWaveNumber = row.WaveBullNumber
            BullWaveType = row.WaveBull
        elif (BullWaveType != row.WaveBull):            
            fig.add_annotation(x=row.Index, y=getattr(row,'Close'),text=BullWaveType, showarrow=False, yshift=10,font=dict(color='cyan'))            
            BullWaveType = row.WaveBull            
        if (BearWaveNumber < row.WaveBearNumber):            
            fig.add_annotation(x=row.Index, y=getattr(row,'Close'),text=BearWaveType, showarrow=False, yshift=-10, font=dict(color='yellow'))
            BearWaveNumber = row.WaveBearNumber
            BearWaveType = row.WaveBear
        elif (BearWaveType != row.WaveBear):       
            fig.add_annotation(x=row.Index, y=getattr(row,'Close'),text=BearWaveType, showarrow=False, yshift=-10, font=dict(color='yellow'))            
            BearWaveType = row.WaveBear        

        #if (trade_counter < row.trade_number_start):
        #    fig.add_annotation(x=row.Index, y=getattr(row,'total_cum_pnl'),yref="y2",text=row.trade_number_start, showarrow=False, yshift=10, font=dict(color='green'))      
        #    trade_counter = row.trade_number_start
    
    
    fig.update_layout(
        title=f'{currency} MACD Results',
        template='plotly_dark', 
        margin=dict(t=35,b=5,l=5,r=5),
        yaxis_title='PnL',
        yaxis2_title='Price',
        xaxis_title='Dates',
        legend_title='',
        width=1000,
        height=500,
    )
    
    #fig.for_each_trace(lambda t: t.update(line=dict(color=t.marker.color)))
    #fig.for_each_trace(lambda trace: trace.update(name=trace.name.replace("total_cum_pnl", "Cumulative PnL")))
    fig = go.FigureWidget(fig)

    
    return fig 

In [ ]:
def build_trade_grid(df, currency):        
    
    col_name = 'Close'
    
    percent_renderer = ipd.TextRenderer(format=".1%",horizontal_alignment="center")
    float_renderer = ipd.TextRenderer(format=".2s",horizontal_alignment="right")
    fx_renderer = ipd.TextRenderer(format=".4f", horizontal_alignment="center")
    int_renderer = ipd.TextRenderer(format=".0f", horizontal_alignment="center")
    date_formatter = ipd.TextRenderer(format="%Y/%m/%d", format_type="time", horizontal_alignment="center")
    text_formatter = ipd.TextRenderer(horizontal_alignment="center")
    
    enum_renderer = ipd.TextRenderer(text_value=ipd.VegaExpr(
            """
               if (cell.value == 1, 
                   'Buy', 
                   if(cell.value == -1, 
                       'Sell', 
                       'None')
                    )                   
            """), horizontal_alignment="center")
    
    header_renderer = ipd.TextRenderer(
        text_wrap=True,
        vertical_alignment="top",
        background_color="rgb(50,50,50)",
        horizontal_alignment="center",
    )

    renderers = {  
        "DATE" : date_formatter,
        col_name : fx_renderer,
        "trade_signal" : enum_renderer,
        "WaveBull": text_formatter,
        "WaveBear": text_formatter,         
    }

    column_widths = {
        "DATE" : 65,
        col_name : 65,
        "trade_signal": 70,
        "WaveBull": 65,
        "WaveBear": 85,         
    }

    grid = ipd.DataGrid(
        df[[col_name, 'trade_signal','WaveBull','WaveBear']],
        renderers=renderers,
        header_renderer=header_renderer,
        column_widths=column_widths,
        base_row_header_size=220,
        base_column_header_size=40,
        layout={"width": "770px"},
    )
    return grid

In [ ]:
def build_grid(df):        
    
    percent_renderer = ipd.TextRenderer(format=".1%",horizontal_alignment="center")
    float_renderer = ipd.TextRenderer(format=".2s",horizontal_alignment="right")
    float_no_nan_renderer = ipd.TextRenderer(text_value=ipd.VegaExpr(
            """
            if(
                !isValid(cell.value),
                ' ',
                if(
                    cell.value==0,
                    ' ',
                    format(cell.value, ',.2s')
                )
            )
            """))
    
    int_renderer = ipd.TextRenderer(format=".0f", horizontal_alignment="center")
    date_formatter = ipd.TextRenderer(format="%Y/%m/%d", format_type="time", horizontal_alignment="center")
    text_formatter = ipd.TextRenderer(format=".0s", horizontal_alignment="center")
    
    header_renderer = ipd.TextRenderer(
        text_wrap=True,
        vertical_alignment="top",
        background_color="rgb(50,50,50)",
        horizontal_alignment="center",
    )

    renderers = {        
        "Trade Number": int_renderer,
        "BuySell": text_formatter,
        "Total PnL": float_renderer,
        "Average PnL": float_renderer,
        "Stdev PnL" : float_no_nan_renderer,
        "Maximum Drawdown" : float_renderer,
        "Start Date" : date_formatter,
        "End Date" : date_formatter,   
    }

    column_widths = {
        "Trade Number": 65,
        "BuySell": 65,
        "Total PnL": 85,
        "Average PnL": 85,
        "Stdev PnL" : 85,
        "Maximum Drawdown" : 85,
        "Start Date" : 80,
        "End Date" : 80,   
    }

    grid = ipd.DataGrid(
        df,
        renderers=renderers,
        header_renderer=header_renderer,
        column_widths=column_widths,
        base_row_header_size=220,
        base_column_header_size=40,
        layout={"width": "770px"},
    )
    return grid

In [ ]:
def run(event=None):
    
    spinner.layout.visibility = "visible"
    
    exception_box.children = []
    fig_box.children = []
    grid_box.children = []
    
    try: 
        
        currency = currency_dropdown.value
        
        start_date = start_date_picker.value    
               
        df = get_dataframe(currency, start_date)       
        
        waves = demark_waves.createWaves(df[['Open', 'High', 'Low', 'Close']])        
              
        my_data = df.join(waves[['WaveBull', 'WaveBullNumber', 'WaveBear', 'WaveBearNumber']])
                
        #trade_data, trade_info = createMACDsignals(currency, my_data, 12, 26, 9, start_date=start_date, max_blocks = 10)   
        trade_data, trade_info = createMACDsignals(currency, my_data, 7, 15, 6, start_date=start_date, max_blocks = 5)   
               
        fig = build_chart(trade_data, currency, start_date)
       
        grid = build_grid(trade_info.tail(22))
        
        trade_grid = build_trade_grid(trade_data.tail(1), currency)
        
        fig_box.children = [fig]
        grid_box.children = [grid]
        trade_box.children = [trade_grid]
    except Exception as e:
        exception_box.children = [widgets.Label(f"Error: {e}")]
        
    spinner.layout.visibility = "hidden"

In [ ]:
# Build widgets

today = datetime.datetime.today()
ref_dt = today - relativedelta(years=1) 

currency_dropdown = widgets.Dropdown(
    description="Currency:",
    value='USDZAR',
    options=fx_pairs,
    layout={"width":"170px"},
    style={"description_width": "70px"},
)

start_date_picker = widgets.DatePicker(
    description="Start Date:", 
    value=ref_dt,
    layout={"width":"215px"},
    style={"description_width": "85px"}
)


go_button = widgets.Button(
    description="Run",
    style={"description_width": "initial"},
    layout={"width": "60px", "margin": "3px 5px 5px 10px"},
    button_style="success",
)

go_button.on_click(run)

spinner = widgets.HTML(
    """<i class="fa fa-spinner fa-spin" style="font-size: 18px"></i>""",
    layout={
        "visibility": "hidden", "margin": "3px 0 0 10px", "width": "40px"
    }
)

fig_box = widgets.HBox()
grid_box = widgets.HBox()
trade_box = widgets.HBox()
exception_box = widgets.VBox()

In [ ]:
ui_display = widgets.VBox([
    widgets.HBox([
        currency_dropdown,        
        start_date_picker,
        go_button, 
        spinner
    ]),
    widgets.HBox([
        grid_box,
        fig_box,        
    ]),
    trade_box,
    exception_box
])
# Run on startup
run()

In [ ]:
ui_display